In [3]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [4]:
llm = OpenAI()
chat_model = ChatOpenAI()
llm.predict("hi!")

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..


KeyboardInterrupt: 

In [8]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [11]:
api.dataset_download_file('kaggle/meta-kaggle/download?datasetVersionNumber=1131',
                          file_name='Competitions.csv',
                          path='./')

True

In [ ]:
import zipfile

zip_path = './Competitions.csv.zip'  # Chemin vers le fichier .zip
extract_path = './'  # Chemin où extraire le fichier .csv

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)